> This is one of the 100 recipes of the [IPython Cookbook](http://ipython-books.github.io/), the definitive guide to high-performance scientific computing and data science in Python.


# 3.2. Converting an IPython notebook to other formats with nbconvert

You need pandoc, a LateX distribution, and the Notebook dataset on the book's website. On Windows, you also need pywin32 (`conda install pywin32` if you use Anaconda).

1. Let's open the test notebook in the `data` folder. A notebook is just a plain text file (JSON), so we open it in text mode (`r` mode).

In [3]:
with open('data/test.ipynb', 'r') as f:
    contents = f.read()
print(len(contents))

3787


In [4]:
print(contents[:345] + '...' + contents[-33:])

{
 "metadata": {
  "celltoolbar": "Edit Metadata",
  "name": "",
  "signature": "sha256:50dbdf439f8e74ec064d182ebfdb541a873e6608ee11f807e625253d8b02c459"
 },
 "nbformat": 3,
 "nbformat_minor": 0,
 "worksheets": [
  {
   "cells": [
    {
     "cell_type": "markdown",
     "metadata": {},
     "source": [
      "# First chapter"
     ]
    },
  ...
   ],
   "metadata": {}
  }
 ]
}


2. Now that we have loaded the notebook as a string, let's parse it with the `json` module.

In [5]:
import json
nb = json.loads(contents)

3. Let's have a look at the keys in the notebook dictionary.

In [6]:
print(nb.keys())
print('nbformat ' + str(nb['nbformat']) + 
      '.' + str(nb['nbformat_minor']))

dict_keys(['metadata', 'nbformat', 'nbformat_minor', 'worksheets'])
nbformat 3.0


The version of the notebook format is indicated in `nbformat` and `nbformat_minor`.

3. The main field is `worksheets`: there is only one by default. A worksheet contains a list of cells, and some metadata.

In [7]:
nb['worksheets'][0].keys()

dict_keys(['metadata', 'cells'])

4. Each cell has a type, optional metadata, some contents (text or code), possibly one or several outputs, and other information. Let's look at a Markdown cell and a code cell.

In [8]:
nb['worksheets'][0]['cells'][1]

{'cell_type': 'markdown',
 'metadata': {'my_field': ['value1', '2405']},
 'source': ["Let's write some *rich* **text** with [links](http://www.ipython.org) and lists:\n",
  '\n',
  '* item1\n',
  '* item2\n',
  '    1. subitem\n',
  '    2. subitem\n',
  '* item3']}

In [9]:
nb['worksheets'][0]['cells'][2]

{'cell_type': 'code',
 'collapsed': False,
 'input': ['import numpy as np\n',
  'import matplotlib.pyplot as plt\n',
  '%matplotlib inline\n',
  'plt.figure(figsize=(2,2));\n',
  "plt.imshow(np.random.rand(10,10), interpolation='none');\n",
  "plt.axis('off');\n",
  'plt.tight_layout();'],
 'language': 'python',
 'metadata': {},
 'outputs': [{'metadata': {},
   'output_type': 'display_data',
   'png': 'iVBORw0KGgoAAAANSUhEUgAAAP8AAAEMCAYAAAALeWDJAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz\nAAAWJQAAFiUBSVIk8AAABo9JREFUeJzt3E2opnUdxvHrec4RGtLUiF5mCKOXOTSoCWHEkFSkhQS1\nKMWFSNbUtDFyIUWgVtJGhMQQjKKacFFEm1mJELSIQNo4YPZyFgUugmrobXSKOWfuFjFg2zM/OA7X\n57O/L/7neZ4v91n9V8uyLAHqrPf7AMD+ED+UEj+UEj+UEj+UEj+UEj+UEj+UEj+UEj+UEj+UEj+U\nEj+UEj+UEj+U2tzvA7zc4a1nsr19dnTzrbuvG927e3396F6SPJATo3uP5K7RvSS596Or8c2Nb+yO\n7p3/+fy77MzxjdG9K67eGd1br5PdF/f47OhJgEuG+KGU+KGU+KGU+KGU+KGU+KGU+KGU+KGU+KGU\n+KGU+KGU+KGU+KGU+KGU+KGU+KGU+KGU+KHUK+oOv9f/9k05k3Ojm9svHB7d28xTo3tJcv/q1tG9\n+56fvRsvSe796/xPZbl19l7AjQfn/+4fverjo3snX7p5dO9/

5. Once parsed, the notebook is represented as a Python dictionary. Manipulating it is therefore quite convenient in Python. Here, we count the number of Markdown and code cells.

In [10]:
cells = nb['worksheets'][0]['cells']
nm = len([cell for cell in cells
          if cell['cell_type'] == 'markdown'])
nc = len([cell for cell in cells
          if cell['cell_type'] == 'code'])
print(("There are {nm} Markdown cells and "
       "{nc} code cells.").format(
        nm=nm, nc=nc))

There are 2 Markdown cells and 1 code cells.


6. Let's have a closer look at the image output of the cell with the matplotlib figure.

In [11]:
png = cells[2]['outputs'][0]['png']
cells[2]['outputs'][0]['png'] = png[:20] + '...' + png[-20:]
cells[2]['outputs'][0]

{'metadata': {},
 'output_type': 'display_data',
 'png': 'iVBORw0KGgoAAAANSUhE...ErAAAAAElF\nTkSuQmCC\n',
 'text': ['<matplotlib.figure.Figure at 0x747c080>']}

In general, there can be zero, one, or multiple outputs. Besides, each output can have multiple representations. Here, the matplotlib figure has a PNG representation (the base64-encoded image) and a text representation (the internal representation of the figure).

7. Now, we are going to use nbconvert to convert our text notebook to other formats. This tool can be used from the command-line (if you are using IPython < 4.x, replace command `jupyter` with `ipython`). Here, we convert the notebook to an HTML document.

In [12]:
!jupyter nbconvert --to html data/test.ipynb

[NbConvertApp] Converting notebook data/test.ipynb to html
[NbConvertApp] Writing 193028 bytes to test.html


8. Let's display this document in an `<iframe>` (a small window showing an external HTML document within the notebook).

In [13]:
from IPython.display import IFrame
IFrame('test.html', 600, 200)

9. We can also convert the notebook to LaTeX and PDF. In order to specify the title and author of the document, we need to *extend* the default LaTeX template. First, we create a file `mytemplate.tplx` that extends the default `article.tplx` template provided by nbconvert. We precise the contents of the author and title blocks here.

In [14]:
%%writefile mytemplate.tplx
((*- extends 'article.tplx' -*))

((* block author *))
\author{Cyrille Rossant}
((* endblock author *))

((* block title *))
\title{My document}
((* endblock title *))

Writing mytemplate.tplx


10. Then, we can run nbconvert by specifying our custom template.

In [ ]:
!jupyter nbconvert --to latex --template mytemplate data/test.ipynb
!pdflatex test.tex

[NbConvertApp] Converting notebook data/test.ipynb to latex
[NbConvertApp] Support files will be in test_files/
[NbConvertApp] Making directory test_files
[NbConvertApp] Writing 14866 bytes to test.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.16 (TeX Live 2015) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./test.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 21 languages loaded.
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/graphics/trig.sty)
(/usr/local/texlive/2015basic/texmf-dist/tex/latex/latexconfig